# Regularization Sandbox
This notebook is a test-bed for regularization and reconstruction methods

In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

# Load motiondeblur module and Dataset class
import libwallerlab.projects.motiondeblur as md
from libwallerlab.utilities.io import Dataset, isDataset

# Platform imports
import os, glob

# Debugging imports
import llops as yp
import matplotlib.pyplot as plt

yp.config.setDefaultBackend('arrayfire')

## Load Data

In [2]:
# Define user for path setting
dataset_path = '/Users/zfphil/datasets/motiondeblur/'

# Define which dataset to use
dataset_type = 'coded'
dataset_label =  'res'

# Find files in this directory
folder_list = glob.glob(os.path.join(dataset_path, '*/'))
dataset_list = [folder for folder in folder_list if isDataset(folder)]

# Filter datasets in directory
filtered_dataset_list = [folder_name for folder_name in folder_list if (dataset_type in folder_name) and (dataset_label in folder_name)]
assert not len(filtered_dataset_list) > 1, "More than one dataset with criterion found!"
assert not len(filtered_dataset_list) == 0, "No dataset with criterion found!"

# Create dataset object (loads metadata)
dataset = Dataset(filtered_dataset_list[0])

# Force type to be motiondeblur
dataset.metadata.type = 'motiondeblur'

# Select green channel
dataset.channel_mask = [0]

# Preprocess dataset (md-specific)
md.preprocess(dataset)

Loading 1 frames...


/Users/zfphil/.conda/lib/python3.7/site-packages/arrayfire/array.py:157: ComplexWarning: Casting complex values to real discards the imaginary part
  c_val = c_double_t(val)


# Create Reconstruction Object

In [3]:
# Set position segment
dataset.frame_mask = [20,21,22]

# Create recon object
recon = md.recon.Reconstruction(dataset, alpha_blend_distance=0)

# Normalize Measurements
scale = recon.normalize_measurements()

# Register Measurements
offsets = recon.register_measurements(method='xc', preprocess_methods=['normalize', 'highpass'], axis=1, debug=False, write_results=True)
print(offsets)

Loading 3 frames...
[array([0., 0.]), array([0., 0.]), array([0., 0.])]


## Stitch Measurements Together

In [ ]:
recon.reconstruct(iteration_count=-1, mode='static')
recon.show()

## Perform Unregularized Sequential Recon

In [ ]:
recon.reconstruct(iteration_count=100, step_size=1, mode='sequential')
recon.show()

## Perform Unregularized Global Recon

In [ ]:
recon.reconstruct(iteration_count=100, step_size=1, mode='global')
recon.show()

## Perform Single-Frame Recon

In [ ]:
recon.reconstruct(iteration_count=100, step_size=1, mode='single', frame_number=1, reg_types={'l2': 1e-6})
recon.show()

## TV Regularization

In [ ]:
recon.reconstruct(iteration_count=10, step_size=1, reg_types={'tv':1e-1})
recon.show()

recon.save('/Users/zfphil/Desktop/', filename='tv', formats=['png'])

## L2 Regularization

In [ ]:
import prox_tv
import numpy as np

In [ ]:
'dr' - Douglas Rachford splitting.
'pd' - Proximal Dykstra splitting.
'yang' - Yang’s algorithm.
'condat' - Condat’s gradient.
'chambolle-pock' - Chambolle-Pock’s gradient.

dr: 8s
pd:7s

In [ ]:
q = prox_tv.tv1_2d(np.asarray(recon.object_recovered), 0.3, method='pd', n_threads=8)

In [ ]:
plt.figure()
plt.imshow(yp.real(q))

In [ ]:
recon.reconstruct(iteration_count=100, step_size=1, reg_types={'l2': 1e-5})
recon.show()

recon.save('/Users/zfphil/Desktop/', filename='l2', formats=['png'])

## TV (Using RED)

In [ ]:
recon.reconstruct(iteration_count=100, step_size=1, reg_types={'tv_wavelet': 1e-1, 'weight': 1.0})
recon.show()

recon.save('/Users/zfphil/Desktop/', filename='tv_bregman', formats=['png'])

## Bilateral Filter (Using RED)

In [ ]:
recon.reconstruct(iteration_count=100, step_size=1, reg_types={'bilateral': 1e-2})
recon.show()

recon.save('/Users/zfphil/Desktop/', filename='bilateral', formats=['png'])

## Median Filter (Using RED)

In [ ]:
recon.reconstruct(iteration_count=100, step_size=1e-5, reg_types={'median': 1e-1})
recon.show()

recon.save('/Users/zfphil/Desktop/', filename='median', formats=['png'])

## Wavelet Sparsity

In [ ]:
recon.reconstruct(iteration_count=400, step_size=1, reg_types={'wavelet': 1e-2})
recon.show()

recon.save('/Users/zfphil/Desktop/', filename='wavelet', formats=['png'])

## Native Sparsity

In [ ]:
recon.reconstruct(iteration_count=100, step_size=1, reg_types={'l1': 1e-1})
recon.show()

recon.save('/Users/zfphil/Desktop/', filename='l1', formats=['png'])

## Fourier Sparsity

In [ ]:
recon.reconstruct(iteration_count=100, step_size=1e-3, reg_types={'l1F': 1e-2})
recon.show()

recon.save('/Users/zfphil/Desktop/', filename='l1', formats=['png'])